In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Liver_cirrhosis/GSE150734'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Prognostic Liver Signature profiles in biopsy tissues from non-alcoholic fatty liver disease (NAFLD) patients in the U.S."
!Series_summary	"Background/Aims: There is a major unmet need to assess prognostic impact of anti-fibrotics in clinical trials due to the slow rate of liver fibrosis progression. We aimed to develop a surrogate biomarker to predict future fibrosis progression.  Methods: A Fibrosis Progression Signature (FPS) was defined to predict fibrosis progression within 5 years in HCV and NAFLD patients with no to minimal fibrosis at baseline (n=421), and validated in an independent NAFLD cohort (n=78). The FPS was used to assess response to 13 candidate anti-fibrotics in organotypic ex vivo cultures of clinical fibrotic liver tissues (n=78), and cenicriviroc in NASH patients enrolled in a clinical trial (n=19, NCT02217475). A serum-protein-based surrogate FPS (FPSec) was developed and technically evaluated in a liver disease patient coho

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Check for gene expression data
is_gene_available = True  # This dataset likely contains gene expression data

# Step 2.1: Data Availability
# Based on the sample characteristics dictionary, it's not clear if 'Liver_cirrhosis', 'age,' and 'gender' data are available
trait_row = 0  # Inferring liver cirrhosis might be represented by fibrosis stage
age_row = None  # No clear indication of age data
gender_row = None  # No clear indication of gender data

# Step 2.3: Data Type Conversion
def convert_trait(value):
    value = value.split(':')[1].strip()
    if value == '0':
        return 0
    elif value == '1':
        return 1
    else:
        return None

def convert_age(value):
    # Age data is not provided
    return None

def convert_gender(value):
    # Gender data is not provided
    return None

# Step 3: Save Metadata
save_cohort_info('GSE150734', './preprocessed/Liver_cirrhosis/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Liver_cirrhosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Liver_cirrhosis/trait_data/GSE150734.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4557370': [0], 'GSM4557371': [0], 'GSM4557372': [0], 'GSM4557373': [0], 'GSM4557374': [0], 'GSM4557375': [0], 'GSM4557376': [0], 'GSM4557377': [0], 'GSM4557378': [0], 'GSM4557379': [0], 'GSM4557380': [0], 'GSM4557381': [0], 'GSM4557382': [1], 'GSM4557383': [0], 'GSM4557384': [0], 'GSM4557385': [0], 'GSM4557386': [0], 'GSM4557387': [0], 'GSM4557388': [1], 'GSM4557389': [1], 'GSM4557390': [0], 'GSM4557391': [0], 'GSM4557392': [0], 'GSM4557393': [0], 'GSM4557394': [1], 'GSM4557395': [0], 'GSM4557396': [1], 'GSM4557397': [0], 'GSM4557398': [1], 'GSM4557399': [0], 'GSM4557400': [0], 'GSM4557401': [1], 'GSM4557402': [0], 'GSM4557403': [0], 'GSM4557404': [0], 'GSM4557405': [0], 'GSM4557406': [0], 'GSM4557407': [0], 'GSM4557408': [1], 'GSM4557409': [1], 'GSM4557410': [1], 'GSM4557411': [1], 'GSM4557412': [1], 'GSM4557413': [1], 'GSM4557414': [1], 'GSM4557415': [1], 'GSM4557416': [0], 'GSM4557417': [0], 'GSM4557418': [0], 'GSM4557419': [0], 'GSM4557420': [0], 'GSM4557421': [0], 'GSM4557422

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['AARS', 'ABLIM1', 'ACOT2', 'ACSM3', 'ACTR2', 'ADD3', 'ADH5', 'ADH6',
       'ADRA2B', 'AEBP1', 'AKAP13', 'AKR1A1', 'AKR1D1', 'ALAS1', 'ALDH9A1',
       'ANKRD46', 'ANXA1', 'ANXA3', 'AOX1', 'AP1B1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# Since requires_gene_mapping is False, we skip STEP6.
# Printing "Skipping STEP6 as requires_gene_mapping is False."
print("Skipping STEP6 as requires_gene_mapping is False.")


Gene annotation preview:
{'ID': ['FSHB', 'SH3GL2', 'RBM34', 'NCAPH', 'EGF'], 'Gene symbol': ['FSHB', 'SH3GL2', 'RBM34', 'NCAPH', 'EGF'], 'Category': ['Signature gene', 'Signature gene', 'Signature gene', 'Signature gene', 'Signature gene'], 'Associated phenotype': ['Poor prognosis/high HCC risk', 'Poor prognosis/high HCC risk', 'Poor prognosis/high HCC risk', 'Poor prognosis/high HCC risk', 'Poor prognosis/high HCC risk'], 'order': [1.0, 2.0, 3.0, 4.0, 5.0], 'EntrezGeneID': [2488.0, 6456.0, 23029.0, 23397.0, 1950.0], 'Alternative symbol': ['FSHB', 'SH3GL2', 'RBM34', 'NCAPH', 'EGF'], 'GB_ACC': ['NM_000510.2', 'NM_003026.2', 'NM_015014.2', 'NM_015341.3', 'NM_001963.4']}
Skipping STEP6 as requires_gene_mapping is False.


### Step 7: Data Normalization and Merging

In [7]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Liver_cirrhosis/gene_data/GSE150734.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, merged_data = judge_and_remove_biased_features(merged_data, 'Liver_cirrhosis')

# 4. Save the cohort information.
cohort = 'GSE150734'
json_path = './preprocessed/Liver_cirrhosis/cohort_info.json'
save_cohort_info(cohort, json_path, True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Liver_cirrhosis/GSE150734.csv'
    merged_data.to_csv(csv_path)


For the feature 'Liver_cirrhosis', the least common label is '1.0' with 61 occurrences. This represents 19.74% of the dataset.
The distribution of the feature 'Liver_cirrhosis' in this dataset is fine.

